In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import requests 
import re
import networkx as nx

# Set Up Sequencing Data
1. Load Excel data into Pandas.
2. Create lists to isolate columns in various ways.


In [ ]:
def load_data(filepath,sheet):
    df = pd.read_excel(filepath,sheet_name=sheet)
    
    return df

df = load_data('published-data/media-2.xlsx','epi_data_matrix')

# Rename the first column to something useful
df.rename(columns={'Unnamed: 0':'Gene'},inplace=True)

# Isolate groups making up the cochlea floor
# Use list comprehension to join suffix to groups
mean_suffix = '_Mean'
diff_suffix = '_Avg_diff'
all_groups = ['10.1.0','10.1.1','10.1.2','10.1.3','10.2.0','10.2.1','10.3.0','10.3.1',\
             '10.3.2','10.4','10.5','22.0','22.1','22.2.0','22.2.1','22.3',\
             '24.0','24.1']
early_floor_groups = ['24.0','24.1','10.0','10.1.0','10.1.1','10.4','10.2.1','10.3.0']
cycling_floor_groups = ['10.0','24.0','24.1']

all_group_means = [group + mean_suffix for group in all_groups]
early_floor_group_means = [group + mean_suffix for group in early_floor_groups]
cycling_floor_group_means = [group + mean_suffix for group in cycling_floor_groups]

all_group_diffs = [group + diff_suffix for group in all_groups]
early_floor_group_diffs = [group + diff_suffix for group in early_floor_groups]
cycling_floor_group_diffs = [group + diff_suffix for group in cycling_floor_groups]

# Get just the columns we're interested (average differential expression)
df[['Gene']+all_group_diffs].head()

# Retrieve sequencing info by gene name.

In [ ]:
df[['Gene']+all_group_diffs].\
loc[df['Gene'] == 'Bmp4']

# Set Up for Querying STRING

## Initialize

In [ ]:
# Retrieve STRING version
string_api_url = 'https://string-db.org/api'
output_format = 'json'
mus_musculus = 10090 # https://www.ncbi.nlm.nih.gov/Taxonomy/TaxIdentifier/tax_identifier.cgi
request_url = '/'.join([string_api_url,output_format,'version'])
r = requests.post(request_url)
current_string_version_info = r.json()[0]
current_string_version_info

## Function to retrieve standardized STRING IDs for gene names

In [ ]:
def get_string_ids(gene_names_list):
    ## Tested: working 2020-09-20
    
    assert type(gene_names_list) is list,"First argument must be a list."
    
    method = 'get_string_ids'
    request_url = '/'.join([string_api_url,output_format,method])
    params = {
        'identifiers' : '\r'.join(gene_names_list),
        'species' : mus_musculus,
        'limit' : 1
    }
    
    r = requests.post(request_url,data=params)
    gene_id_json = r.json()
    
    # Convert into dataframe
    
    string_id_df = pd.DataFrame.from_records(gene_id_json)        
    
    return string_id_df

## Function to retrieve protein interactions from STRING for a gene list

In [ ]:
def get_string_interaction_partners(gene_names_list,score):
    
    assert type(gene_names_list) is list,"First argument must be a list."
    assert (score <= 999 and score >= 0),"Score must be in the range 0 to 999."
    
    string_id_df = get_string_ids(gene_names_list)
    
    string_ids = string_id_df['stringId'].tolist()
    
    method = 'interaction_partners'
    request_url = '/'.join([string_api_url,output_format,method])
    params = {
        'identifiers' : '\r'.join(string_ids),
        'species' : mus_musculus,
        'required_score' : score
    }
    
    r = requests.post(request_url,data=params)
    partners_json = r.json()
    
    partners_df = pd.DataFrame.from_records(partners_json)
    
    return partners_df

## Demo interaction network query

In [ ]:
genes = ['Jag1','Bmp4','Sox2']
score = 970
partners_df = get_string_interaction_partners(genes,score)

partners_df.sort_values(by=['preferredName_A','score'],ascending=False).head(30)

# Cross-Reference scRNA-Seq with STRING-DB
### For each group:
1. Create a set of nodes for transcripts that are present (i.e. log-normalized mean expression > threshold).
2. Query STRING for interaction partners of all present transcripts.
3. If A and B partners from STRING are present in the group, create an edge between them.
4. Add edges to network according to As and Bs that are present.
5. Query STRING for networks of all present transcripts.
6. Determine if a path between any present transcripts exists in the network.
7. If a path exists, determine whether intermediate nodes are confirmed not present.

In [ ]:
# 1. Create a set of nodes for transcripts that are present (i.e. log-normalized mean expression > threshold).

# Let's start with just one group. And maybe let's do this from the main dataframe with all data.
mean_thresh = 0.8

for g in all_group_means:
    group = ['Gene',g]
    g_df = df[group].loc[df[g] > mean_thresh]

    G = nx.Graph()
    G.add_nodes_from(g_df['Gene'].to_list())
    nx.draw(G,with_labels=True)
    plt.title(g + ' Log-Norm Expression Nodes, Threshold = ' + str(mean_thresh))
    plt.show()

### Uncomment above to plot graphs

In [ ]:
# 2. Query STRING for interaction partners of all present transcripts.

group_no = 1

group = ['Gene',all_group_means[group_no]]
print(group)
g_df = df[group].loc[df[all_group_means[group_no]] > mean_thresh]
gene_list = g_df['Gene'].tolist()
score = 700
gene_interactions_df = get_string_interaction_partners(gene_list,score)
gene_interactions_df.head()
#gene_interactions_stringIds_list = gene_interactions_df['stringId_B'].tolist()




In [ ]:
# 3. If A and B partners from STRING are present in the group, create an edge between them
# Get a list of the stringId codes for genes in step 1.
network_nodes_list = g_df['Gene'].to_list()
network_stringIds_df = get_string_ids(network_nodes_list)
network_stringIds_list = network_stringIds_df['stringId'].to_list()

# Add a column to the interaction data frame for 'B is present'
gene_interactions_df['B_is_present']=False

# Check if each 'stringId_B' from gene_interactions_df is present anywhere in network_stringIds_df['stringId']
network_stringIds_df['stringId']

idx = 0
for id in gene_interactions_df['stringId_B'].tolist():
    tf = network_stringIds_df['stringId'].str.contains(id)
    total_occurence = tf.sum()
    if(total_occurence > 0):
        gene_interactions_df['B_is_present'].iloc[idx] = True
    idx = idx + 1
    
sequence_network_df = gene_interactions_df.loc[gene_interactions_df['B_is_present'] == True].reset_index()
sequence_network_df

In [ ]:
# 4. Add edges to network according to As and Bs that are present.
sequence_network_df

list_A = sequence_network_df['preferredName_A'].tolist()
list_B = sequence_network_df['preferredName_B'].tolist()

idx = 0
for gene_A in list_A:
    G.add_edge(list_A[idx],list_B[idx])
    print(list_A[idx] + '\t\t--\t' + list_B[idx])
    idx = idx + 1

# nx.draw(G,with_labels=True)
deg = G.degree()

to_remove = [n for (n,deg) in deg if deg < 1]

G.remove_nodes_from(to_remove)

pos = nx.spring_layout(G,k=100,iterations=100)

nx.draw(G,with_labels=True,node_size = 100)

plt.title(all_group_means[group_no] + ' Log-Norm Expression Nodes, Threshold = ' + str(mean_thresh))
plt.show()

In [ ]:
S = [G.subgraph(c).copy() for c in nx.connected_components(G)]

for sub_graph in S:
    nx.draw(sub_graph,with_labels=True,node_size = 100)
    plt.title(all_group_means[group_no] + ' Log-Norm Expression Nodes, Threshold = ' + str(mean_thresh))
    plt.show()

In [ ]:
## Retrieve upregulated genes for a group...
## Within a group, what genes are upregulated
diff_expr_threshold = 0.5

col_temp = all_group_diffs[0]
group_temp = ['Gene']+[col_temp]
group_temp_df = df[group_temp]
upreg_df = group_temp_df.loc[group_temp_df[all_group_diffs[0]] > diff_expr_threshold].\
sort_values(by=col_temp,ascending=False)

## Retrieve interaction partners for each of these genes
## and see if any of these partners are in the upregulated list
## Within the set of upregulated genes, what proteins interact with them?
upreg_gene_list= upreg_df['Gene'].tolist()
score = 980
partners_df = get_string_interaction_partners(upreg_gene_list,score)

## Within the interaction partners of a protein, which are included in the 
## upregulated genes set?

# Compare preferredName_B' or 'stringId_B' in partners_df to 'Gene' in upreg_df


# New column in partners_df will indicate whether proteinB is upregulated
# 1. Create list of True/False called upreg_list
# 2. parters_df['Upregulated'] = upreg_list # This creates the new column
upreg_bool_list = [False for i in range(n_rows)]
upreg_bool_list

idx = 0;
for upreg_gene in upreg_gene_list:
    # Test if this upregulated gene shows up in the partners dataframe
    bool_findings = partners_df['preferredName_B'].str.contains(upreg_gene)
    total_occurence = bool_findings.sum()
    if(total_occurence > 0):
        upreg_bool_list[idx] = True
        
    idx = idx + 1
    
partners_df['Upregulated'] = upreg_bool_list
partners_df

# for upreg_gene in upreg_gene_list
#     boolean_findings = parters_df['preferredName_A'].str.contains(upreg_gene)
#     print(boolean_findings)
    
#cycling_floor_group_diffs[0]

# for d in all_group_diffs:
#     # In each group filter for genes that show high expression
#     group_df = df[['Gene']+[d]].loc[df[d] > diff_expr_threshold].reset_index()
    
#     # For the genes that are highly expressed, 
#     # identify 1st shell interacting genes with 'highest' confidence (>0.9)
    
#     # Isolate the gene names from the current group into a temporary list 
#     group_genes = []
#     for idx, genes in group_df.iterrows():
#         group_genes.append(genes['Gene'])
        
#     # Map identifiers to STRING standards

    
# # genes_stringId_temp
